In [7]:
# Imports

import datetime
# import itertools
from itertools import permutations
import math
import os
import pickle
import platform
import random
from tkinter import Tk

# from cvxopt import solvers, matrix
from matplotlib import animation
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import torch
# import torchvision
# from torchvision import transforms, models,datasets
# from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# from tools import *
import tools

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# OS related settings
if platform.system() == 'Windows':
    print('Windows')
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')

macOS
Interactive plot activated


In [15]:
# Save

FM_by_dim_all = {}
MSEs_seen_by_dim_all = {}
MSEs_unseen_by_dim_all = {}

for file_name in os.listdir('output/'):
    print(file_name)
    print('output/' + file_name)
    with open('output/' + file_name, 'rb') as f:
        FM_by_dim = pickle.load(f)
        MSEs_seen_by_dim = pickle.load(f)
        MSEs_unseen_by_dim = pickle.load(f)
        for k in FM_by_dim.keys():
            if k in FM_by_dim_all.keys():
                FM_by_dim_all[k] = np.append(FM_by_dim_all[k], FM_by_dim[k], axis=0)
                MSEs_seen_by_dim_all[k] = np.append(MSEs_seen_by_dim_all[k], MSEs_seen_by_dim[k], axis=0)
                MSEs_unseen_by_dim_all[k] = np.append(MSEs_unseen_by_dim_all[k], MSEs_unseen_by_dim[k], axis=0)
            else:
                FM_by_dim_all[k] = FM_by_dim[k]
                MSEs_seen_by_dim_all[k] = MSEs_seen_by_dim[k]
                MSEs_unseen_by_dim_all[k] = MSEs_unseen_by_dim[k]
        
        



# Save
now = datetime.datetime.now().strftime("-%m-%d-%Y@%H.%M.%S")
with open('output/' + 'All_output_in_one' + now, 'wb') as f:
    pickle.dump(FM_by_dim_all, f)
    pickle.dump(MSEs_seen_by_dim_all, f)
    pickle.dump(MSEs_unseen_by_dim_all, f)
    
FM_by_dim = FM_by_dim_all
MSEs_seen_by_dim = MSEs_seen_by_dim_all
MSEs_unseen_by_dim = MSEs_unseen_by_dim_all



dim_list = FM_by_dim.keys()
num_per_perm_list_train = [10, 50, 100]
model_names = ['QP']

ChI_saved_file-11-08-2021@15.14.24
output/ChI_saved_file-11-08-2021@15.14.24
All_output_in_one-11-08-2021@15.16.25
output/All_output_in_one-11-08-2021@15.16.25


In [10]:
avg_funcs = [np.amin, np.amax, np.mean, None, None, None, None] # List of avg functions

weights = {3: np.asarray([[0.8, 0.1, 0.1],
                          [0.2, 0.7, 0.1],
                          [0.5, 0.2, 0.3],
                          [0.1, 0.3, 0.5]]),
           4: np.asarray([[0.1, 0.2, 0.3, 0.4],
                          [0.2, 0.2, 0.2, 0.4],
                          [0, 0.1, 0.3, 0.6],
                          [0, 0, 0.4, 0.6]]),
           5: np.asarray([[0.1, 0.1, 0.1, 0.2, 0.5],
                          [0, 0, 0.4, 0.3, 0.3],
                          [0, 0, 0, 0.4, 0.6],
                          [0.1, 0.8, 0.1, 0, 0]]),
           6: np.asarray([[0.1, 0.1, 0.1, 0.1, 0.2, 0.4],
                          [0, 0, 0.3, 0.3, 0.3, 0.1],
                          [0, 0, 0, 0, 0.4, 0.6],
                          [0.1, 0.7, 0.1, 0.1, 0, 0]]),}

avg_names = {}
for dim in dim_list:
    weight = weights[dim]
    weight_legend = []
    for w in weight:
        weight_legend.append(' '.join(map(str, (w*10).astype(int))))
    avg_names[dim] = ['Min', 'Max', 'Mean'] + weight_legend
    
output_dir = 'output_plots/'

In [16]:
MSEs_seen_by_dim[3].shape

(24, 3, 6, 7, 1)

In [17]:
# Plot
plt.close('all')
# Plot for seen
for dim in dim_list:
    MSEs_seen = MSEs_seen_by_dim[dim]
    MSE_mean = np.mean(MSEs_seen, 0)
    MSE_max = np.max(MSEs_seen, 0)
    MSE_min = np.min(MSEs_seen, 0)
    
    num_perm = math.factorial(dim)
    
    for npp_idx in range(len(num_per_perm_list_train)):
        for model_idx in range(len(model_names)):
            fig, ax = plt.subplots()
            x = (np.arange(MSE_mean.shape[1])+1) / MSE_mean.shape[1]
            plt.plot(x, MSE_mean[npp_idx, :, :, model_idx])
            ax.set_title('MSE (seen), Model=' + model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]))
            
            ax.legend(avg_names[dim])
            
            ax.set_xlabel('Percentage of Seen Data')
            ax.set_ylabel('MSEs avg')
            ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

            for avg_idx in range(len(avg_funcs)):
                plt.fill_between(x, MSE_min[npp_idx, :, avg_idx, model_idx], MSE_max[npp_idx, :, avg_idx, model_idx], alpha=0.1)
                
            plt.savefig(output_dir + model_names[model_idx] + ' Dim=' + str(dim) + ' NPP=' + str(num_per_perm_list_train[npp_idx]) + ' (MSE seen).png')
                
                
# Plot for unseen
for dim in dim_list:
    MSEs_unseen = MSEs_unseen_by_dim[dim]
    MSE_mean = np.mean(MSEs_unseen, 0)
    MSE_max = np.max(MSEs_unseen, 0)
    MSE_min = np.min(MSEs_unseen, 0)
    
    num_perm = math.factorial(dim)

    for npp_idx in range(MSE_mean.shape[0]):
        for model_idx in range(MSE_mean.shape[-1]):
            fig, ax = plt.subplots()
            x = (np.arange(MSE_mean.shape[1])+1) / (MSE_mean.shape[1]+1)
            plt.plot(x, MSE_mean[npp_idx, :, :, model_idx])
            ax.set_title('MSE (unseen), Model=' + model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]))
            
            ax.legend(avg_names[dim])
            
            ax.set_xlabel('Percentage of Seen Data')
            ax.set_ylabel('MSEs avg')
            ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

            for avg_idx in range(len(avg_funcs)):
                plt.fill_between(x, MSE_min[npp_idx, :, avg_idx, model_idx], MSE_max[npp_idx, :, avg_idx, model_idx], alpha=0.1)
                
            plt.savefig(output_dir + model_names[model_idx] + ' Dim=' + str(dim) + ' NPP=' + str(num_per_perm_list_train[npp_idx]) + ' (MSE unseen).png')

In [5]:
# Video for FM change



for dim in dim_list:
    FM = FM_by_dim[dim]
    FM_mean = np.mean(FM, 0)
    FM_min = np.amin(FM, 0)
    FM_max = np.amax(FM, 0)
    
    avg_name = avg_names[dim]
    fm_targets = [tools.get_min_fm_target, tools.get_max_fm_target, tools.get_mean_fm_target, tools.get_w_avg_target(weights[dim][0]), tools.get_w_avg_target(weights[dim][1]), tools.get_w_avg_target(weights[dim][2]), tools.get_w_avg_target(weights[dim][3])]
    
    
    for npp_idx in range(len(num_per_perm_list_train)):
        for model_idx in range(len(model_names)):
            for avg_idx, fm_target in enumerate(fm_targets):
                fm_mean = FM_mean[npp_idx, :, avg_idx, model_idx, :]
                fm_min = FM_min[npp_idx, :, avg_idx, model_idx, :]
                fm_max = FM_max[npp_idx, :, avg_idx, model_idx, :]

                # First set up the figure, the axis, and the plot element we want to animate
                fig = plt.figure()
                ax = plt.axes(xlim=(0, np.size(fm_mean, -1)-1), ylim=(-0.1, 1.1))
                line, = ax.plot([], [], lw=2)
                ax.set_xlabel('Fuzzy Measure Key')
                ax.set_ylabel('Fuzzy Measure Value')
                ax.set_title(model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]) + ' ' + avg_name[avg_idx])

                # initialization function: plot the background of each frame
                def init():
                    x = list(range(np.size(fm_mean, -1)))
                    y = list(fm_target(dim).values())
                    plt.plot(x, y)
                    ax.legend(['FM Target'], loc=4)

  
                # animation function.  This is called sequentially
                def animate(i):
                    x = np.asarray(list(range(np.size(fm_mean, -1))))
                    y = fm_mean[i, :]
                    line.set_data(x, y)
                    ax.legend(['FM Predict (Seen data percentage ' + str("{0:.0%}".format((i+1)/np.size(fm_mean, 0))) + ')', 'FM Target'])
                    ax.collections = []
                    plt.fill_between(x, fm_min[i, :], fm_max[i, :], color='blue', alpha=0.1)
                    return line,

                # call the animator.  blit=True means only re-draw the parts that have changed.
                anim = animation.FuncAnimation(fig, animate, frames=np.size(fm_mean, 0), init_func=init(), interval=200, blit=True)

                # save the animation as an mp4.  This requires ffmpeg or mencoder to be
                # installed.  The extra_args ensure that the x264 codec is used, so that
                # the video can be embedded in html5.  You may need to adjust this for
                # your system: for more information, see
                # http://matplotlib.sourceforge.net/api/animation_api.html
                anim.save(output_dir + model_names[model_idx] + ', Dim=' + str(dim) + ', NPP=' + str(num_per_perm_list_train[npp_idx]) + ' w_AVG=' + avg_name[avg_idx] + '.mp4', fps=2)
        #         plt.show()
                plt.close('all')

In [ ]:
# 3D Plot #pP vs %
plt.close('all')
# Plot for seen
for dim in dim_list:
    MSEs_seen = MSEs_seen_by_dim[dim]
    MSE_mean = np.mean(MSEs_seen, 0)
    MSE_max = np.max(MSEs_seen, 0)
    MSE_min = np.min(MSEs_seen, 0)
    
    num_perm = math.factorial(dim)
    
#     for npp_idx in range(len(num_per_perm_list_train)):
    for model_idx in range(len(model_names)):
        for func_idx in range(len(avg_names[dim])):
        
            x = (np.arange(MSE_mean.shape[1])+1) / MSE_mean.shape[1]
            y = np.asarray(list(range(MSE_mean.shape[0])))
            y_str = [str(i) for i in num_per_perm_list_train]

            X, Y = np.meshgrid(x, y)
            Z = MSE_mean[:, :, func_idx, model_idx]

            fig, ax = plt.subplots()
            ax = plt.axes(projection='3d')

            ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
            ax.set(yticks=range(len(y)), yticklabels=y_str) 
            ax.set_xlabel('% of permutations')
            ax.set_ylabel('# per permutation')
            ax.set_zlabel('MSE mean');
            ax.set_title('MSE (seen) #pP vs %, Model=' + model_names[model_idx] + ', Dim=' + str(dim) + ', Avg func=' + avg_names[dim][func_idx])
            plt.savefig(output_dir + '3D-' + model_names[model_idx] + ' Dim=' + str(dim) + ' Func=' + avg_names[dim][func_idx] + ' (MSE seen).png')
                
# Plot for unseen
for dim in dim_list:
    MSEs_unseen = MSEs_unseen_by_dim[dim]
    MSE_mean = np.mean(MSEs_unseen, 0)
    MSE_max = np.max(MSEs_unseen, 0)
    MSE_min = np.min(MSEs_unseen, 0)
    
    num_perm = math.factorial(dim)

#     for npp_idx in range(MSE_mean.shape[0]):
    for model_idx in range(MSE_mean.shape[-1]):
        for func_idx in range(len(avg_names[dim])):
            
            x = (np.arange(MSE_mean.shape[1])+1) / (MSE_mean.shape[1]+1)
            y = np.asarray(list(range(MSE_mean.shape[0])))
            y_str = [str(i) for i in num_per_perm_list_train]

            X, Y = np.meshgrid(x, y)
            Z = MSE_mean[:, :, func_idx, model_idx]

            fig, ax = plt.subplots()
            ax = plt.axes(projection='3d')

            ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
            ax.set(yticks=range(len(y)), yticklabels=y_str) 
            ax.set_xlabel('% of permutations')
            ax.set_ylabel('# per permutation')
            ax.set_zlabel('MSE mean');
            ax.set_title('MSE (unseen) #pP vs %, Model=' + model_names[model_idx] + ', Dim=' + str(dim) + ', Avg func=' + avg_names[dim][func_idx])
            plt.savefig(output_dir + '3D-' + model_names[model_idx] + ' Dim=' + str(dim) + ' Func=' + avg_names[dim][func_idx] + ' (MSE unseen).png')

<ipython-input-59-775cd6e32c65>:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


In [ ]:
plt.close('all')